**Goals**

1. Get this thing organized and working in numpy 
1. If really really needed do it in JAX

In [1]:
import numpy as np
import numpy.linalg as linalg
from sklearn.linear_model import Lasso
from copy import deepcopy, copy 
from typing import NamedTuple
from sklearn.exceptions import ConvergenceWarning
from sklearn.utils._testing import ignore_warnings

## Utils

In [10]:
def get_psd(sigma):
    M, v = np.linalg.eig(sigma)
    v_inv = np.linalg.inv(v) 
    E = np.diag(np.clip(M,0,np.inf))+0.0001
    return (v@E@v_inv).astype(float)    

def get_obj_fct(lasso, hyper):
    #Something is wrong here...#TODO very wrong
    @ignore_warnings(category=ConvergenceWarning)
    def obj_fct_n(Theta, client_codeword):
        #This needs fiving
        obj_fct = 0.0
        for nn in range(hyper.n):
            for t in range(client_codeword[nn].shape[0]):
                y = client_codeword[nn][t]
                lasso.fit(Theta.T, y) # involves solving a LASSO problem
                hj = lasso.coef_
                obj_fct += 0.5*linalg.norm(y - hj@Theta )**2 + hyper.lmbd*linalg.norm( hj, 1)
        return (obj_fct + 0.5*hyper.eta* linalg.norm( Theta )**2)
    return obj_fct_n

def quant(x, s):
    #TODO
    return x
    # return (1/s) * np.sign(x) * linalg.norm(x) * np.floor(s* np.abs(x)/(linalg.norm(x))* 
    #                                                       np.random.uniform(size=x.shape))
   
    
class Hyper(NamedTuple):
    ## Problems dimensions
    d = 15 # original dimension
    m = 10 # codeword dimension
    n = 20 # number of workers
    T = 50 # Samples per worker
    homo_switch = 0 #0 for heter data, 1 for homo
    #Loss function parameters
    eta = 0.1 #dict l2 penalty weight
    lmbd = 0.05 #codeword l1 penalty weight
    rep_times = 1 # no. of repetition to run the experiment
    # parameters for the algorithms 
    BatchSize = 3 # batch size per client per run
    Participate = 5 # no of active workers each run
    alpha = 0.01 # step size
    squant = 15 # quantization level
    IterNum = 10_000 # no. of iterations per experiment
    
def generate_data(hyper:Hyper):
    shared_dict = np.random.normal(size=(hyper.d,hyper.m))
    client_codeword , client_data = [], [] 
    for nn in range(hyper.n): 
        client_data.append(np.random.normal(size=(hyper.T,hyper.d))* np.random.binomial(
            1, 0.2,size=(hyper.T,hyper.d)))
        client_codeword.append(client_data[nn]@shared_dict + 0.0001*np.random.normal(size=(hyper.T,hyper.m)))
    if hyper.homo_switch == 0:
        return shared_dict,client_codeword,client_data
    else:
        raise NotImplementedError
    

## Experiment

In [11]:
hyper = Hyper()
true_dict, client_codeword, client_data = generate_data(hyper)
lasso = Lasso(alpha=hyper.lmbd)
init_dict,_,_ = generate_data(hyper)
## Take care of homogenous experiment
if hyper.homo_switch == 1:
    client_codeword = np.stack(client_codeword)
    client_codeword = [client_codeword for i in range(hyper.n)]    
lasso = Lasso(alpha=hyper.lmbd, fit_intercept=False)
obj_fct_n = get_obj_fct(lasso, hyper)

p = (hyper.Participate/hyper.n)
obj_sto_all = []

## The Code

In [13]:
@ignore_warnings(category=ConvergenceWarning)
def fit_client(x,y):
    fitted = lasso.fit(x,y)
    return fitted.coef_

for rep in range(hyper.rep_times):
    Theta = deepcopy(init_dict)
    V_SS1 = [np.zeros((hyper.d,hyper.d)) for _ in range(hyper.n)] 
    V_SS2 = [np.zeros((hyper.m,hyper.d)) for _ in range(hyper.n)] 
    Msg_SS1 = [np.zeros((hyper.d,hyper.d)) for _ in range(hyper.n)] 
    Msg_SS2 = [np.zeros((hyper.m,hyper.d)) for _ in range(hyper.n)] 
    Va_SS1, Ha_SS1, Sa_SS1 = (np.zeros((hyper.d,hyper.d)) for _ in range(3))
    Va_SS2, Ha_SS2, Sa_SS2 = (np.zeros((hyper.m,hyper.d)) for _ in range(3))
    
    obj_sto = []
    print("rep = ", rep, " \n")
    for t in range(hyper.IterNum):
        if t%50 == 0:
            obj_sto.append(obj_fct_n(Theta, client_codeword))
            print( "t ",t ," obj = ",  obj_sto[-1])
        
        idx_PP = np.random.permutation(hyper.n)[:hyper.Participate]
        #for each client
        for nn in idx_PP:
            #batch data
            if hyper.homo_switch == 0:
                idx_batch = np.random.permutation(hyper.T)[:hyper.BatchSize]
                yy = client_codeword[nn][idx_batch]
            else:
                idx_batch = np.random.permutation(hyper.n*hyper.T)[:hyper.BatchSize]
                yy = client_codeword[nn][idx_batch] 

            #client message
            SS1 = np.zeros((hyper.d,hyper.d))
            SS2 = np.zeros((hyper.m,hyper.d))
            for b in range(hyper.BatchSize):
                hj = fit_client(Theta.T, yy[b])
                SS1 += np.outer(hj,hj)
                SS2 += np.outer(yy[b],hj)

            ##Message conostruction
            Delta1 = SS1/hyper.BatchSize - Sa_SS1 - V_SS1[nn]
            Delta2 = SS2/hyper.BatchSize - Sa_SS2 - V_SS2[nn]
            V_SS1[nn] = V_SS1[nn] + (hyper.alpha/p)*quant(Delta1, hyper.squant)
            V_SS2[nn] = V_SS2[nn] + (hyper.alpha/p)*quant(Delta2, hyper.squant)
            Msg_SS1[nn] = quant(Delta1, hyper.squant)
            Msg_SS2[nn] = quant(Delta2, hyper.squant); 
        

        #Server Update
        step_size = 0.1 / np.sqrt(t+0.1)
        D_SS1 = (1/(hyper.n*p))*sum([Msg_SS1[i] for i in idx_PP])
        D_SS2 = (1/(hyper.n*p))*sum([Msg_SS2[i] for i in idx_PP])    
        Ha_SS1 = Va_SS1 + D_SS1
        Ha_SS2 = Va_SS2 + D_SS2
        Sa_SS1 = Sa_SS1 + step_size*Ha_SS1
        Sa_SS1 = get_psd(0.5*(Sa_SS1+Sa_SS1.T))
        Sa_SS2 = Sa_SS2 + step_size*Ha_SS2
        Va_SS1 = Va_SS1 + (hyper.alpha/(hyper.n*p))*sum([Msg_SS1[i] for i in idx_PP])
        Va_SS2 = Va_SS2 + (hyper.alpha/(hyper.n*p))*sum([Msg_SS2[i] for i in idx_PP])
        Theta = Sa_SS2 @ ( np.linalg.inv(Sa_SS1 + hyper.eta*np.eye(hyper.d)))
        Theta = Theta.T  

    obj_sto_all.append(obj_sto)

rep =  0  

t  0  obj =  548.6716599563402
t  50  obj =  5005.081203617387
t  100  obj =  4804.797465901892
t  150  obj =  4706.078982069541
t  200  obj =  4639.49533560454
t  250  obj =  4590.220034095256
t  300  obj =  4547.093872687393
t  350  obj =  4517.994190280377
t  400  obj =  4479.862910436956
t  450  obj =  4457.314116300466
t  500  obj =  4435.111770821811
t  550  obj =  4409.709638007107
t  600  obj =  4391.158443994672
t  650  obj =  4378.728611980151
t  700  obj =  4365.452426345228
t  750  obj =  4347.641757064082
t  800  obj =  4335.227056873515
t  850  obj =  4327.049755821042
t  900  obj =  4317.1582885716125
t  950  obj =  4306.719990488061
t  1000  obj =  4296.280376202852
t  1050  obj =  4288.528878056763
t  1100  obj =  4282.211153368312
t  1150  obj =  4272.1460725037105
t  1200  obj =  4266.599284169972
t  1250  obj =  4266.095236612214
t  1300  obj =  4262.530511334075
t  1350  obj =  4257.222047025555
t  1400  obj =  4255.691035860642
t  1450  obj =  4256.340

KeyboardInterrupt: 